# Decoradores y fábricas de decoradores

Contacto:
adelfino@gmail.com
Telegram: @adelfino

## Cimientos

Las funciones pueden ser argumentos:

In [ ]:
help(print)

Las funciones pueden ser valores de retorno:

In [ ]:
def funcion():
    return print

print(funcion())

Las funciones pueden ser enlazadas a nuevos nombres:

In [ ]:
funcion = print
funcion('Hello World')

Se puede recibir argumentos posicionales o por keyword de forma variable con los operadores * (que creará una tupla para los argumentos posicionales) y ** (que creará un diccionario para los argumentos por keyword)

In [ ]:
def funcion(*args, **kwargs):
    print(args, kwargs)
    
funcion(1, 2, uno=1, dos=2)

Las funciones pueden ser anidadas:

In [ ]:
def madre():
    def hija():
        print('Hello World')

    hija()
    print('This is getting old...')

madre()

try:
    hija()
except NameError as e:
    print('Excepción:', e)

Los nombres accedidos en las funciones anidadas que no se hayan definido en ellas serán buscados en la función que la anida, y su valor será preservado en una "closure".

In [ ]:
import inspect

nombre_global = 'valor global'

def madre():
    nombre_no_local = 'valor no local' #desde la perspectiva de hija()

    def hija():
        print('Global:', nombre_global)
        print('No local:', nombre_no_local)
        print('Unbound:', nombre_unbound)
        print()

    return hija

funcion = madre()

print(inspect.getclosurevars(funcion), '\n')

print('Ejecutamos la función hija sin cambiar nada del entorno')
try:
    funcion()
except NameError as e:
    print('Excepción:', e, '\n')
    pass

print('Ejecutamos tras crear el nombre global nombre_unbound')
nombre_unbound = 1
funcion()

print('Ejecutamos tras cambiar valor de nombre global')
nombre_global = '123'
funcion()

## Decoradores

Alteran la ejecución de una función, una corutina o una clase. Generalmente llaman al objeto decorado en algún momento, pero es totalmente opcional.

### Decoradores de funciones, y corutinas

Suelen retornar un nuevo objeto.

In [ ]:
def decorador(obj):
    def objeto_decorado(*args, **kwargs):
        print('Hola')
        r = obj(*args, **kwargs)
        print('Chau')
        return r

    return objeto_decorado

sum = decorador(sum)
x = sum([1, 2, 3])
print(x)

### Decoradores de métodos de instancia

Suelen retornar un nuevo objeto.

In [ ]:
def decorador(obj):
    def objeto_decorado(self, *args, **kwargs):
        self.cuenta += 1

    return objeto_decorado

class Clase:
    def __init__(self):
        self.cuenta = 0

    def contar(self):
        self.cuenta += 1

    contar = decorador(contar)
   
instancia = Clase()
instancia.contar()
instancia.cuenta

### Decoradores de clase

Suelen retornar el mismo objeto.

In [ ]:
def decorador(obj):
    def __str__(self):
        return 'Hola'

    obj.__str__ = __str__

    return obj

class Clase:
    pass

Clase = decorador(Clase)
instancia = Clase()

print(instancia)

### Preservar metadata y referencia cuando se retorna un nuevo objeto

In [ ]:
import re

def decorador(obj):
    def objeto_decorado(*args, **kwargs):
        return obj(*args, **kwargs)

    return objeto_decorado

re.search = decorador(re.search)

for attrib in ('__module__', '__name__', '__qualname__', '__annotations__', '__doc__'):
    print(f'{attrib}:', getattr(re.search, attrib))

In [ ]:
import functools
import re

def decorador(obj):
    def objeto_decorado(*args, **kwargs):
        return obj(*args, **kwargs)

    objeto_decorado = functools.update_wrapper(objeto_decorado, obj)

    return objeto_decorado

re.findall = decorador(re.findall)

for attrib in ('__module__', '__name__', '__qualname__', '__annotations__', '__doc__'):
    print(f'{attrib}:', getattr(re.findall, attrib))
    
print()
print('Original:', re.findall.__wrapped__)
print('Decorado:', re.findall)

## Sintaxis de decoración en tiempo de definición

In [ ]:
def decorador1(obj):
    print('Aplicando decorador 1')
    return obj

def decorador2(obj):
    print('Aplicando decorador 2')
    return obj

#obj = decorador1(decorador2(obj))

@decorador2
@decorador1
def obj():
    print('Ejecutando obj')

obj()

## Fábricas de decoradores

In [ ]:
import datetime

def fabrica_de_decoradores(formato):
    def decorador(obj):
      def objeto_decorado(*args, **kwargs):
          timestamp = datetime.datetime.today()
          print('{:{}} Inicio'.format(timestamp, formato))

          r = obj(*args, **kwargs)

          timestamp = datetime.datetime.today()
          print('{:{}} Final'.format(timestamp, formato))

          return r

      return objeto_decorado
    
    return decorador

@fabrica_de_decoradores(formato='%Y%m%dT%M%H%S')
def funcion():
   print('Test')

funcion()